In [0]:
cd '/content/drive/My Drive/PD/cis-pd'

/content/drive/My Drive/PD/cis-pd


In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import scale
import scipy.io as sio
import pickle

In [0]:
label_train = pd.read_csv('data_labels/CIS-PD_Training_Data_IDs_Labels.csv')

In [0]:
label_train['dyskinesia'].value_counts(dropna = False)

0.0    688
NaN    670
1.0    252
2.0    168
3.0     73
4.0      7
Name: dyskinesia, dtype: int64

In [0]:
label_train = pd.read_csv('data_labels/CIS-PD_Training_Data_IDs_Labels.csv')
label_test  = pd.read_csv('data_labels/CIS-PD_Testing_Data_IDs_Labels.csv')

memo ={}
#Return a single wo minutes data
def id_single_data(subject_id,i):
    measurement_id = label_train[label_train['subject_id'] == subject_id]['measurement_id'].values
    id = measurement_id[i]
    gy = pd.read_csv('training_data/'+id+'.csv')    
    gy['step'] = gy['t'].map(lambda x:int(x*5)/10)
    gy = gy.groupby('step').mean()

    return gy[["X","Y","Z"]].values


#Return an array of each subject_id with or without scaling within each data points
#seperate data by [0,0,0]
def concatData(subject_id,z = False):
    measurement_id = label_train[label_train['subject_id'] == subject_id]['measurement_id'].tolist()
    s = len(measurement_id)
    measurement_id += label_test[label_test['subject_id'] == subject_id]['measurement_id'].tolist()
    group = []
    i = 0
    print("train")
    folder = 'training_data'
    for id in measurement_id:
        if i == s:
          print("test")
          folder = 'testing_data'
        try:
            gy = pd.read_csv(folder+'/'+id+'.csv')
        except:
            i+=1
            continue
        gy['step'] = gy['Timestamp'].map(lambda x:int(x*5)/10)
        gy = gy.groupby('step').mean()


        if not z:
            group.extend(gy[["X","Y","Z"]].values)
        else:
            group.extend(scale(gy[["X","Y","Z"]].values,axis = 0))
        
        group.append([0,0,0])
        i += 1
    
    return np.array(group)

#Return an array of each subject with scaling within each person
#First scaling, then add [0,0,0] to split the data

def zscoreAmongperson(subject_id):
    memo[subject_id] = {'train':[],'test':[]}
    measurement_id = label_train[label_train['subject_id'] == subject_id]['measurement_id'].tolist()
    s = len(measurement_id)
    measurement_id += label_test[label_test['subject_id'] == subject_id]['measurement_id'].tolist()
    l = []
    group = []
    y_train=[]
    test_id = []
    i = 0
    mm = memo[subject_id]['train']
    folder = 'training_data'
    for id in measurement_id:
        
        if i == s:
          print("test")
          folder = 'testing_data'
          mm = memo[subject_id]['test']

        try:
            gy = pd.read_csv(folder+'/'+id+'.csv')
            if i < s:
              y_train.append(label_train[label_train['measurement_id'] == id].iloc[:,2:].values)
            else:
              test_id.append(id)

        except:
            print(id[:8]," not in smart folder")
            i+=1
            continue
        gy['step'] = gy['Timestamp'].map(lambda x:int(x*5)/10)
        gy = gy.groupby('step').mean()
        
        l.append(len(gy))
        mm.append(len(gy))
        group.extend(gy[["X","Y","Z"]].values)
        i += 1
    group = scale(np.array(group),axis = 0)
    cur = 0
    for i in l:
        cur += i
        group = np.insert(group,cur,np.array([[0,0,0]]),axis = 0)
        cur += 1
    
    return np.array(group),y_train,test_id


def createArray(l):
    noScale = []
    Scaledatapoints = []
    Scaleperson = []
    y_train = []
    test_id = []

    for i,id in enumerate(l):

        noScale.append(concatData(id,False))
        np.savetxt('outputA_Total/'+str(id_list[i])+'.csv',noScale[-1])
        #sio.savemat('outputA/'+str(id_list[i])+'.mat',{'data':noScale[-1]})
        Scaledatapoints.append(concatData(id,True))
        #sio.savemat('outputB/'+str(id_list[i])+'.mat',{'data':Scaledatapoints[-1]})
        np.savetxt('outputB_Total/'+str(id_list[i])+'.csv',Scaledatapoints[-1])
        A = zscoreAmongperson(id)
        Scaleperson.append(A[0])
        #sio.savemat('outputB/'+str(id_list[i])+'.mat',{'data':Scaleperson[-1]})
        np.savetxt('outputC_Total/'+str(id_list[i])+'.csv',Scaleperson[-1])
        y_train.extend(A[1])
        test_id.extend(A[2])
        print(id," done!")
    

    return np.array(noScale),np.array(Scaledatapoints),np.array(Scaleperson),y_train,test_id
#np.argmax(B,axis = 1)
#np.max(C,axis = 1)with open("real-pd-memo.pkl", "wb") as file: pickle.dump(memo, file)
#np.min(C,axis = 1)


id_list = label_train['subject_id'].unique()


In [0]:
zscoreAmongperson(1004)

In [0]:
id_list

array([1004, 1006, 1007, 1019, 1020, 1023, 1032, 1034, 1038, 1039, 1043,
       1044, 1046, 1048, 1049, 1051])

In [0]:
A,B,C,y_train,test_id = createArray(id_list)

train
test
train
test
test
1004  done!
train
test
train
test
test
1006  done!
train
test
train
test
test
1007  done!
train
test
train
test
test
1019  done!
train
test
train
test
test
1020  done!
train
test
train
test
test
1023  done!
train
test
train
test
test
1032  done!
train
test
train
test
test
1034  done!
train
test
train
test
test
1038  done!
train
test
train
test
test
1039  done!
train
test
train
test
test
1043  done!
train
test
train
test
test
1044  done!
train
test
train
test
test
1046  done!
train
test
train
test
test
1048  done!
train
test
train
test
test
1049  done!
train
test
train
test
test
1051  done!


In [0]:
test_id


['0e80673b-8c90-474f-8388-00fdf25795e4',
 'a7b61d40-9f26-4878-a2d7-bd766b8ae6f5',
 'fc851afd-133c-4e8f-8deb-0079cc442358',
 '90c2c320-1007-47b4-9f8e-cdd37067dfb2',
 '379fd23b-e38f-4214-8a70-7a829004359f',
 '6877b35f-474a-41b6-ab7b-d9fdcd3155ca',
 '9e37d451-afba-494e-a531-e1c459c65a55',
 'e94eee19-58a8-485e-b7e6-627ef92f5d32',
 '22f6b2b7-d4e6-4066-a2de-0e2f3a62ee65',
 '9afe4f12-18cf-4457-9f15-d90528b7d0e2',
 '3185e61a-ea2f-4d65-a698-12e1e7b9ccb1',
 'b40e9fdb-f9d0-48c9-a45d-49d52bbcad57',
 '2c1367bd-3c90-490b-a04d-1e10a841dbe3',
 'b56726c3-576e-4e8d-85c3-adb471e8eb42',
 '04029adc-f6ab-4f68-b8fd-04076cea597d',
 '83f9b27a-02ff-43b0-a5f7-5005864be79e',
 'b181a5b7-f836-42de-aec5-48d2bbbbcdff',
 '69d06ab0-8013-4e74-b4be-169fd49d0e4c',
 '022feb7c-8c8e-438f-bdf4-c9acd8116527',
 '39d1af70-eb4a-4fa7-a96f-8b7d1697580d',
 'c6577df5-9a7f-480e-8ea7-abd46a3e74de',
 'd2c6d64f-f146-4d00-b0e6-e71b5767c79a',
 'daf3f51b-a409-471f-a99f-84ad0c8a3adf',
 '7598e18a-79c2-4936-a951-b9aedae10d31',
 '3c00152e-4195-

In [0]:

with open("cis-pd-memo.pkl", "wb") as file: 
  pickle.dump(memo, file)

with open("cis-pd-y_train.pkl", "wb") as file: 
  pickle.dump(np.array(y_train).reshape(-1,3),file)

In [0]:
with open("cis-pd_testid.pkl", "wb") as file: 
  pickle.dump(test_id,file)